In [1]:
# import libraries

import os
import pandas as pd
from datetime import date
import pickle

## Using sunscrape to supplement denier tweets ##
- Since 2019-06-01: 'climatechangeisfalse', 'climatechangenotreal', 'climatechangehoax', 'globalwarminghoax' (around 30,000, before hydrating), climatedeniers (6,000), climatehoax (68,000)
- Since 2017-09-21 (missing hashtags): climatechangescam(1484), climatehysteria(11,000), globalwarmingisahoax (774), globalwarmingscam(600), globalcooling(10,000)
- https://firstdraftnews.org/articles/climate-change-misinformation-conspiracy-memes/
- https://awario.com/blog/climate-change-denial/


In [5]:
today = date.today()
end_date = today

In [45]:
search_term = 'globalcooling'
from_date = '2017-09-21'

In [46]:
os.system(f"snscrape --since {from_date} twitter-search '{search_term} until:{end_date}' > result-tweets.txt")

if os.stat("result-tweets.txt").st_size == 0:
    counter = 0
else:
    df = pd.read_csv('result-tweets.txt', names=['link'])
    counter = df.size

print('Number Of Tweets : '+ str(counter))

Number Of Tweets : 10306


In [8]:
climate_hoax = pd.read_csv('result-tweets.txt')

In [12]:
climate_hoax.to_csv('climate_hoax.csv')

In [48]:
filenames = ['climate_false.txt', 'climate_not_real.txt', 'climatechangehoax.txt',
            'climatechangescam.txt', 'climatedeniers.txt', 'climatehoax.txt',
            'climatehysteria.txt', 'globalcooling.txt','globalwarminghoax.txt',
            'globalwarmingisahoax.txt', 'globalwarmingscam.txt']

with open('denier_supplements.txt', 'w') as outfile:
    for fname in filenames:
        with open(fname) as infile:
            for line in infile:
                outfile.write(line)

In [61]:
data = pd.read_csv('denier_supplements.txt', header=None)
data

,0
0,https://twitter.com/RClarkofNC/status/13315671...
1,https://twitter.com/baeareavibez/status/132707...
2,https://twitter.com/blessedpolish/status/13065...
3,https://twitter.com/Kringdom1/status/117585894...
4,https://twitter.com/20Fireblade13/status/11581...
...,...
127799,https://twitter.com/TeamTrumpUSA/status/918519...
127800,https://twitter.com/100monkeys/status/91811357...
127801,https://twitter.com/JohnInLongmont/status/9174...
127802,https://twitter.com/howardbrown7/status/917017...


In [62]:
data1 = data
data1.columns = ['url']
data1

,url
0,https://twitter.com/RClarkofNC/status/13315671...
1,https://twitter.com/baeareavibez/status/132707...
2,https://twitter.com/blessedpolish/status/13065...
3,https://twitter.com/Kringdom1/status/117585894...
4,https://twitter.com/20Fireblade13/status/11581...
...,...
127799,https://twitter.com/TeamTrumpUSA/status/918519...
127800,https://twitter.com/100monkeys/status/91811357...
127801,https://twitter.com/JohnInLongmont/status/9174...
127802,https://twitter.com/howardbrown7/status/917017...


In [ ]:
.iloc[:,[0,1]]

In [65]:
separated = data1['url'].str.split('/', expand = True)

In [67]:
separated.columns = ['one', 'two','three','four','five','six']

In [71]:
ids = separated['six']

In [74]:
ids = ids.drop_duplicates()

In [75]:
ids

0         1331567111735533569
1         1327071840209051648
2         1306503581709807616
3         1175858943668740098
4         1158157717439209475
                 ...         
127799     918519356253986817
127800     918113571095089153
127801     917409761494274048
127802     917017030330671104
127803     914918334080020480
Name: six, Length: 124753, dtype: object

In [76]:
ids.to_csv('denier_ids.txt', header=False, index=False, sep='\t')

## Building training and prediction datasets

In [2]:
pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns', None)

In [4]:
# Import 2017-2019 dataset with climate believer/denier hashtags

hashtags = pd.read_csv('../datasets/tweets_hashtags.csv', low_memory=False)
print('hashtags shape:',hashtags.shape)

# Import 2017-2019 dataset with geotags

geotags = pd.read_csv('../datasets/climate_tweet_ids.csv',low_memory=False)
print('geotags shape:',geotags.shape)

# Import denier supplement dataset

denier_supp = pd.read_csv('../datasets/denier_supp.csv',low_memory=False)
print('denier_supp shape:',denier_supp.shape)

hashtags shape: (587817, 35)
geotags shape: (321316, 35)
denier_supp shape: (124740, 35)


In [5]:
dfs = [hashtags, geotags, denier_supp]

if all([set(dfs[0].columns) == set(df.columns) for df in dfs]):
    print('All have the same columns')
else:
    print('Some have different columns')

All have the same columns


In [6]:
# Concatenate the three datasets

dfs = [hashtags, geotags, denier_supp]
df = pd.concat(dfs, axis = 0).drop_duplicates().reset_index(drop=True)
df.shape

(1033560, 35)

In [7]:
cols = [0,1,3,4,7,8,9,12,13,15,16,17]
df.drop(df.columns[cols], axis=1, inplace=True)


df = df.drop(df.loc[:,'tweet_url':'user_listed_count'].columns, axis = 1)
df = df.drop(df.loc[:, 'user_name':'user_verified'].columns, axis = 1)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1033560 entries, 0 to 1033559
Data columns (total 8 columns):
 #   Column          Non-Null Count    Dtype 
---  ------          --------------    ----- 
 0   hashtags        801703 non-null   object
 1   favorite_count  1033560 non-null  int64 
 2   id              1033560 non-null  int64 
 3   lang            1033560 non-null  object
 4   place           332208 non-null   object
 5   retweet_count   1033560 non-null  int64 
 6   text            1033560 non-null  object
 7   user_location   896589 non-null   object
dtypes: int64(3), object(5)
memory usage: 63.1+ MB


In [10]:
df.head()

,hashtags,favorite_count,id,lang,place,retweet_count,text,user_location
0,climatechange climateaction sustainability fb,0,930123883520413697,en,NaN,0,We are not going to get too many more warnings...,"Holmdel, NJ"
1,juice reuse reduce actonclimate,1,965587085993365505,en,NaN,1,I'll be juicing my left over carrots tonight t...,"Miami, FL"
2,Hack4Climate Blockchain ClimateAction ClimateC...,4,928605716159574016,en,NaN,1,In less than 3 days we will #Hack4Climate at @...,"Zürich, Schweiz"
3,climatechange climateaction,1,954288658361802752,en,NaN,1,"""The best way to predict the future is to crea...","Tampere, Finland"
4,protectwhatyoulove climatechangeisreal AnimalR...,0,1012691887440814086,en,NaN,0,Accepting the world for the way it is just mig...,NaN


In [11]:
# Drop non-English tweets

indexes = df[ df['lang'] != 'en' ].index
df.drop(indexes, inplace = True)
  
df.shape

(927487, 8)

In [12]:
# save full dataset

df.to_pickle("../data/full_data.pkl")